# Building a Simple Local AI Voice Assistant

# Requirements

1. Be able to control the voice assistant from voice
2. BE able to talk to it as well as perform commands/actions 
3. Tasks
   1. Create tasks in some task backlog db
   2. Create, read, edit and delete files locally
   3. Send emails
   4. Fully local setup (audio transcription and the AI should be fully local)
   5. Answer questions about personal notes and knowledge management stuff 

In [5]:
# 1 - Voice Control 
# We'll need an audion transcription model to convert audio to text 
# We'll use whisper turbo 3
# source: https://huggingface.co/openai/whisper-large-v3-turbo
# pip install --upgrade pip
# pip install --upgrade transformers datasets[audio] accelerate

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "mps"
torch_dtype = torch.float16

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

result = pipe("./test-audio-file.mp3")
print(result["text"])

/Users/greatmaster/miniconda3/envs/test-env/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


 Olá! Olá! É o Lucas. Fascinante. Eu estou amigo. Olha, quer tomar uma nota? Quer tomar uma nota? A nota é... Pois. Não se esqueça de transferir 247 mil euros para a conta de malta de Lucas Martins. Obrigado. Olá!


In [6]:
# 2 - Interaction using LLM
# We'll use llama 3.2 with Ollama https://ollama.com/
# pip install ollama
# https://github.com/ollama/ollama-python
import ollama
response = ollama.chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.
3. This is because the smaller molecules are more effective at scattering the shorter wavelengths, which have a shorter wavelength and are therefore more easily deflected by the molecular collisions.
4. As a result, the blue light is scattered in all directions and reaches our eyes from all parts of the sky, making it appear blue.

On the other hand, the longer wavelengths of light (such as red and orange) are less affected by the scattering and continue to travel in a more direct path, reaching our eyes from below and above. This is why the s

In [7]:
def get_response(prompt):
    response = ollama.chat(model='llama3.2', 
                           messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content']

get_response("What is the country known for having the best weather in the world?")

'It\'s subjective to determine which country has the "best" weather, as people have different preferences and tolerance levels when it comes to temperature, humidity, sunshine, and other climate-related factors. However, based on various rankings and studies, some of the countries often considered to have the most pleasant or optimal weather include:\n\n1. **Hawaii, USA**: Known for its tropical climate with average temperatures ranging from 70°F (21°C) to 85°F (30°C) throughout the year.\n2. **Costa Rica**: With a tropical climate and two coastlines on the Pacific and Caribbean, Costa Rica offers a warm and sunny weather year-round, averaging 77°F (25°C).\n3. **Spain**: The southern region of Spain, particularly the Canary Islands, have a subtropical oceanic climate with mild winters and pleasant summers.\n4. **New Zealand**: Known for its temperate climate, New Zealand enjoys mild temperatures ranging from 50°F (10°C) in winter to 75°F (24°C) in summer.\n5. **Australia** (specificall

In [8]:
import pyaudio
import wave

def record_audio(filename="prompt.mp3", duration=3, sample_rate=44100, channels=2, chunk=1024):
    """
    Record audio from the microphone and save it to a file.
    
    :param filename: Name of the output file (default: "prompt.mp3")
    :param duration: Duration of the recording in seconds (default: 5)
    :param sample_rate: Sample rate of the recording (default: 44100 Hz)
    :param channels: Number of audio channels (default: 2 for stereo)
    :param chunk: Number of frames per buffer (default: 1024)
    """
    p = pyaudio.PyAudio()

    stream = p.open(format=pyaudio.paInt16,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=chunk)

    print("Recording...")

    frames = []

    for i in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print("Recording finished.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded data as a WAV file
    wf = wave.open(filename.replace('.mp3', '.wav'), 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

    print(f"Audio saved as {filename.replace('.mp3', '.wav')}")

# Example usage:            
record_audio()

Recording...
Recording finished.
Audio saved as prompt.wav


In [9]:
def transcribe(audio_filepath):
    result = pipe(audio_filepath)
    return result["text"]

transcribe("./prompt.wav")

/Users/greatmaster/miniconda3/envs/test-env/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


' What is the meaning of life?'

In [10]:
record_audio()
prompt = transcribe("./prompt.wav")
get_response(prompt)

Recording...
Recording finished.
Audio saved as prompt.wav


/Users/greatmaster/miniconda3/envs/test-env/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


'What a daunting task! Narrowing down the countless amazing books out there to just three is a challenge, but here are some suggestions that have had a significant impact on literature and human understanding:\n\n1. **"To Kill a Mockingbird" by Harper Lee**: This Pulitzer Prize-winning novel has become an American classic, exploring themes of racial injustice, tolerance, and the loss of innocence in a small Alabama town during the 1930s. Through the eyes of Scout Finch, we see the world from a child\'s perspective, gaining insight into the complexities of human nature.\n2. **"The Alchemist" by Paulo Coelho**: This international bestseller tells the story of Santiago, a young shepherd on a quest to fulfill his personal legend and find his treasure. Along the way, he encounters various spiritual teachers, mythical creatures, and surreal events that guide him toward self-discovery and enlightenment.\n3. **"1984" by George Orwell**: Published in 1949, this dystopian novel depicts a chillin

**Tasks**
   1. Create tasks in some task backlog db
   2. Create, read, edit and delete files locally
   3. Send emails
   4. Fully local setup (audio transcription and the AI should be fully local)
   5. Answer questions about personal notes and knowledge management stuff 

In [13]:
# Creating the task db first before writing the tools for the model
import pandas as pd
from datetime import datetime

# Create an empty DataFrame for the tasks database
tasks_df = pd.DataFrame(columns=['task', 'status', 'creation_date', 'completed_date'])

tasks_df

,task,status,creation_date,completed_date


In [23]:
# Tool for adding a task
def add_task(task_description):
    """
    Add a task to the tasks database.
    """
    new_task = pd.DataFrame({
        'task': [task_description],
        'status': ['Not Started'],
        'creation_date': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
        'completed_date': [None]
    })
    global tasks_df
    tasks_df = pd.concat([tasks_df, new_task], ignore_index=True)
    
    return tasks_df

# Tool Calling

Tool calling is about giving LLMs the ability to perform actions.

```
{
    'type': 'function',
    'function': {
        'name': 'create_file',
        'description': 'Create a new file with given content',
        'parameters': {
            'type': 'object',
            'properties': {
                'filename': {
                    'type': 'string',
                    'description': 'The name of the file to create',
                },
                'content': {
                    'type': 'string',
                    'description': 'The content to write to the file',
                },
            },
            'required': ['filename', 'content'],
        },
    },
},
```

In [17]:
tool_add_tasks_to_db = {
    'type': 'function',
    'function': {
        'name': 'add_task',
        'description': 'Add a task to the tasks database',
        'parameters': {
            'type': 'object',
            'properties': {
                'task_description': {
                    'type': 'string',
                    'description': 'The description of the task to add',
                },
            },
            'required': ['task_description'],
        },
    },
}

In [24]:
# Creating tasks in a backlog task db
def get_response_with_tools(prompt):
    response = ollama.chat(model='llama3.2', 
                           messages=[{'role': 'user', 'content': prompt}],
                           tools=[tool_add_tasks_to_db])
    # Process tool calls if present
    if 'tool_calls' in response['message']:
        for tool_call in response['message']['tool_calls']:
            if tool_call['function']['name'] == 'add_task':
                task_description = tool_call['function']['arguments']['task_description']
                add_task(task_description)
                print(f"Task added: {task_description}")
    else:
        return response['message']['content']

In [25]:
get_response_with_tools("Create a task to create a local voice AI assistant")

Task added: Create a local voice AI assistant


In [26]:
tasks_df

,task,status,creation_date,completed_date
0,Create a local voice AI assistant,Not Started,2024-10-20 12:39:31,None


In [42]:
tool_create_file = {
            'type': 'function',
            'function': {
                'name': 'create_file',
                'description': 'Create a new file with given content',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to create',
                        },
                        'content': {
                            'type': 'string',
                            'description': 'The content to write to the file',
                        },
                    },
                    'required': ['filename', 'content'],
                },
            },
        }
tool_read_file = {
            'type': 'function',
            'function': {
                'name': 'read_file',
                'description': 'Read the content of a file',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to read',
                        },
                    },
                    'required': ['filename'],
                },
            },
        }
tool_delete_file = {
            'type': 'function',
            'function': {
                'name': 'delete_file',
                'description': 'Delete a file',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to delete',
                        },
                    },
                    'required': ['filename'],
                },
            },
        }

tool_edit_file = {
            'type': 'function', 
            'function': {
                'name': 'edit_file',
                'description': 'Edit the content of a file',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'filename': {
                            'type': 'string',
                            'description': 'The name of the file to edit',
                        },
                        'content': {
                            'type': 'string',
                            'description': 'The content to write to the file',
                        },
                    },
                    'required': ['filename', 'content'],
                },
            },
        }
tools = [tool_create_file, tool_read_file, tool_delete_file, tool_add_tasks_to_db]

In [43]:
import os
# Writing functions to create, read, edit and delete files

def create_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return f"File {filename} created successfully"

def read_file(filename):
    with open(filename, 'r') as file:
        return file.read()

def edit_file(filename, content):
    with open(filename, 'w') as file:
        file.write(content)
    return f"File {filename} edited successfully"

def delete_file(filename):
    os.remove(filename)
    return f"File {filename} deleted successfully"

# Creating tasks in a backlog task db
def get_response_with_tools(prompt):
    response = ollama.chat(model='llama3.2', 
                           messages=[{'role': 'user', 'content': prompt}],
                           tools=tools)
    # Process tool calls if present
    if 'tool_calls' in response['message']:
        for tool_call in response['message']['tool_calls']:
            if tool_call['function']['name'] == 'add_task':
                task_description = tool_call['function']['arguments']['task_description']
                add_task(task_description)
                print(f"Task added: {task_description}")
            elif tool_call['function']['name'] == 'create_file':
                print("Creating file...")
                filename = tool_call['function']['arguments']['filename']
                content = tool_call['function']['arguments']['content']
                create_file(filename, content)
                print(f"File created: {filename}")
            elif tool_call['function']['name'] == 'read_file':
                print("Reading file...")
                filename = tool_call['function']['arguments']['filename']
                content = read_file(filename)
                print(f"File content: {content}")
            elif tool_call['function']['name'] == 'delete_file':
                print("Deleting file...")
                filename = tool_call['function']['arguments']['filename']
                delete_file(filename)
                print(f"File deleted: {filename}")
    else:
        return response['message']['content']

In [38]:
get_response_with_tools("Create a file called 'test.txt' with the content 'Hello, world!'")

File created: test.txt


In [36]:
tasks_df

,task,status,creation_date,completed_date
0,Create a local voice AI assistant,Not Started,2024-10-20 12:39:31,None
1,"Hello, world!",Not Started,2024-10-20 12:44:55,None
2,"Hello, world!",Not Started,2024-10-20 12:45:59,None
3,create a file directly called,Not Started,2024-10-20 12:46:15,None


In [41]:
record_audio(duration=5)
prompt = transcribe("./prompt.wav")
get_response_with_tools(prompt)

Recording...
Recording finished.
Audio saved as prompt.wav


/Users/greatmaster/miniconda3/envs/test-env/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Creating file...
File created: test.txt


In [44]:
# Save tasks_df to CSV file
tasks_df.to_csv('tasks.csv', index=False)
print("Tasks saved to tasks.csv")

Tasks saved to tasks.csv
